In [ ]:
where_you_keep_project = "neurtech-masters/AI in PWF/"

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Navigate to the desired directory in Google Drive
os.chdir('/content/drive/My Drive/' + where_you_keep_project)

# Verify the current working directory
print("Current Working Directory:", os.getcwd())